### Setup

In [1]:
import sklearn as sk
import numpy as np
import pandas as pd

## Import Data

In [2]:
df = pd.read_csv('/mnt/data/prepared.csv', header=0, index_col=0)
print(len(df), " rows")
print(len(df.columns), " cols")
df.head()

7939  rows
16  cols


,dropperc,KPI_DeviceSat,mins,consecmonths,long,dataperc,KPI_NetworkQual,income,age,bill,datause,calls,education,servcalls,kids,churn_Y
custid,,,,,,,,,,,,,,,,
844336,0.016364,0.016035,550,28,35,0.158936,0.760355,89.2,45,71.44,651,55,11,0,2,0
146041,0.018349,0.017055,545,33,35,0.377930,0.526147,54.2,43,124.23,774,66,9,0,1,0
847745,0.018519,0.008896,378,41,19,0.282227,0.798811,55.3,41,112.42,578,91,12,0,1,0
285565,0.014493,0.016460,552,32,38,0.271240,0.631075,66.8,31,105.12,1111,71,10,0,0,0
754611,0.012132,0.020711,577,4,31,0.198730,0.726428,87.2,43,109.48,407,56,8,0,0,0


### Begin Model Building

In [3]:
columns = list(df.columns)
columns.remove('churn_Y')
y = df["churn_Y"].values
X = df[columns].values

In [4]:
from sklearn.ensemble import AdaBoostClassifier
ad1 = AdaBoostClassifier(learning_rate=1)
ad1 = ad1.fit(X, y)
ad1prb = ad1.predict_proba(X)

In [5]:
from sklearn.ensemble import AdaBoostClassifier
ad2 = AdaBoostClassifier(learning_rate=0.5)
ad2 = ad2.fit(X, y)
ad2prb = ad2.predict_proba(X)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
gb1 = GradientBoostingClassifier(loss = 'exponential', max_depth=3)
gb1 = gb1.fit(X, y)
gb1prb = gb1.predict_proba(X)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb2 = GradientBoostingClassifier(loss = 'exponential', max_depth=10)
gb2 = gb2.fit(X, y)
gb2prb = gb2.predict_proba(X)

In [8]:
from sklearn.ensemble import RandomForestClassifier 
rf1 = RandomForestClassifier(max_depth=None)
rf1 = rf1.fit(X, y)
rf1prb = rf1.predict_proba(X)
rf1pclass = rf1.predict(X)

In [9]:
from sklearn.ensemble import RandomForestClassifier 
rf2 = RandomForestClassifier(max_depth = 5)
rf2 = rf2.fit(X, y)
rf2prb = rf2.predict_proba(X)
rf2pclass = rf2.predict(X)

In [10]:
from sklearn.ensemble import RandomForestClassifier 
rf3 = RandomForestClassifier(max_depth = 30)
rf3 = rf3.fit(X, y)
rf3prb = rf3.predict_proba(X)
rf3pclass = rf3.predict(X)

### Build a Cross Validation Function

In [11]:
from sklearn import cross_validation

# takes a list of models, the input np.array, the target np.array, the type of score to be used with cv, and k
# each element in the list of models should have two items: the model object and the name you want to use for that 
# model object
# returns a dataframe with the names you entered and the mean of the cv scores across all k folds

def cv_fun(models, inputs, target, score, k):
    i = 0
    for m in models:
        scores = cross_validation.cross_val_score(models[i][0], inputs, target, scoring=score, cv=k)
    
        if i==0:
            list1 = list()
            list2 = list()
            
        list1.append(round(scores.mean(),3))
        list2.append(models[i][1])

        i=i+1

    return pd.DataFrame(list1, index=list2, columns=[score])

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Cross Validation on the Training Data

In [12]:
# use the cv function found up under the Setup section
# enter a list with each entry holding the model object followed by a text name you want to give the model

input_models = [[ad1, 'ad1']]
input_models.append([ad2, 'ad2'])
input_models.append([gb1, 'gb1'])
input_models.append([gb2, 'gb2'])
input_models.append([rf1, 'rf1'])
input_models.append([rf2, 'rf2'])
input_models.append([rf3, 'rf3'])

cv_auc = cv_fun(input_models, X, y, 'roc_auc', 5)
cv_acc = cv_fun(input_models, X, y, 'accuracy', 5)
cv_results = cv_auc.join(cv_acc)
cv_results.sort_values(by='roc_auc', ascending=False, inplace=True)

In [13]:
#manually pick best, would be nice to automate this part
cv_results

,roc_auc,accuracy
ad1,0.986,0.976
ad2,0.982,0.969
gb1,0.982,0.971
gb2,0.973,0.971
rf1,0.936,0.962
rf3,0.931,0.962
rf2,0.925,0.952


### Save Best Model to File

In [14]:
import pickle 

# save best model
# best = cv_results.index.values[0]
# file = '/mnt/output/' + best + '.pkl'
pickle.dump('/mnt/output/ad1.pkl', open(file, 'wb')) # w = open for writing, r = open for reading, b = binary

NameError: name 'file' is not defined

In [16]:

#make confusion matrix plot
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot
 
    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix
 
    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']
 
    title:        the text to display at the top of the matrix
 
    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues
 
    normalize:    If False, plot the raw numbers
                  If True, plot the proportions
 
    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph
 
    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
 
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools
 
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
 
    if cmap is None:
        cmap = plt.get_cmap('Blues')
 
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
 
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
 
 
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
 
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.gcf().subplots_adjust(bottom=0.25)
    plt.savefig('/mnt/output/ConfMatx_Best.png', format='png')
    plt.show()
    plt.gcf().clear()
    


In [17]:
from sklearn import metrics

plot_confusion_matrix(cm           = metrics.confusion_matrix(y, ad1.predict(X)), 
                      normalize    = False,
                      target_names = ['no churn', 'churn'],
                      title        = "Confusion Matrix for Best Model")
                      

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/output/ConfMatx_Best.png'

In [19]:
df2 = df
df2["prob"] = ad1.predict_proba(X)[:,1]
df2.to_csv('/mnt/data/modelOut.csv')

In [20]:
df2.head()

,dropperc,KPI_DeviceSat,mins,consecmonths,long,dataperc,KPI_NetworkQual,income,age,bill,datause,calls,education,servcalls,kids,churn_Y,prob
custid,,,,,,,,,,,,,,,,,
844336,0.016364,0.016035,550,28,35,0.158936,0.760355,89.2,45,71.44,651,55,11,0,2,0,0.455417
146041,0.018349,0.017055,545,33,35,0.377930,0.526147,54.2,43,124.23,774,66,9,0,1,0,0.419089
847745,0.018519,0.008896,378,41,19,0.282227,0.798811,55.3,41,112.42,578,91,12,0,1,0,0.417735
285565,0.014493,0.016460,552,32,38,0.271240,0.631075,66.8,31,105.12,1111,71,10,0,0,0,0.446571
754611,0.012132,0.020711,577,4,31,0.198730,0.726428,87.2,43,109.48,407,56,8,0,0,0,0.254516
